**Evaluating GOES images with coincident MODIS and ASTER**

What are the magnitude and distribution of differences between ABI-ASTER & MODIS-ASTER?
* Is there an East-to-West change in differences?
* Is there a North-to-South change in differences?
* Is there a relationship between differences and elevation, slope, or aspect?
* Is there a relationship between differences and fractional vegetation covered area?
* Is there a relationship between differences and fractional snow covered area? (from coincident MODIS)

In [ ]:
import xarray as xr
from xhistogram.xarray import histogram
import richdem as rd
import rioxarray as rxr
import xrspatial as xrs
import numpy as np
import pandas as pd
import geopandas as gpd
import glob
from scipy import stats
import matplotlib.pyplot as plt
%matplotlib inline

import pysolar
import datetime

from asp_binder_utils import get_dem

In [ ]:
import warnings
warnings.filterwarnings('ignore')

Define functions

In [ ]:
def summary_stats(a, b):
    '''Compute summary statistics for the difference between two sets.
    Input two flattened (1-D) arrays with NaN values removed'''
    
    # remove nan values
    a = a[np.isnan(a)==False]
    b = b[np.isnan(b)==False]
    
    # for difference stats
    diff = b - a
    
    # for linear regression stats
    slope, intercept, r_value, p_value, std_err = stats.linregress(a, b)
    
    # populate dict with summary stats
    summary_stats_dict = {
        #'diff' : diff ,
        'n' : len(diff) ,
        'mean_diff' : np.nanmean( diff ),
        'median_diff' : np.nanmedian( diff ),
        'mean_squared_diff' : np.nanmean( diff**2 ),
        'rms_diff' : np.sqrt( np.nanmean( diff**2 ) ),
        'std_diff' : np.nanstd( diff ),
        'slope' : slope,
        'intercept' : intercept,
        'r_value' : r_value,
        'p_value' : p_value,
        'std_err' : std_err
        }
    
    return summary_stats_dict

In [ ]:
def plot_regression_confidence_intervals(_x,_y,conf):

    # linear regression
    slope, intercept, r, p, se = stats.linregress(_x, _y)
    
    # predict y values of origional data using the fit
    p_y = slope*_x + intercept
    
    # calculate the y-error (residuals)
    y_err = _y - p_y
    
    # create series of new test x-values to predict for
    p_x = np.arange(np.min(_x),np.max(_x)+1,1)
    
    # now calculate confidence intervals for new test x-series
    mean_x = np.mean(_x)                 # mean of x
    n = _x.size                          # number of samples in original fit
    t = stats.t.ppf(conf, n-2)            # find the appropriate t value (for n-2, and two tailed 95%)        
    sse = np.sum(y_err**2)              # sum of the squares of the residuals
    st_err = np.sqrt(sse/(n-2))         # standard error
    
    sigma = st_err**2 * (1 + 1/n + ( ( n*(p_x-mean_x)**2 ) / ( n*np.sum(_x**2) - np.sum(_x)**2 ) ) )
    confs = t * np.sqrt(sigma)
    
    # now predict y based on test x-values
    p_y = slope*p_x + intercept
    
    # get lower and upper confidence limits based on predicted y and confidence intervals
    lower = p_y - np.abs(confs)
    upper = p_y + np.abs(confs)
    
    # plot line of best fit
    c_x = np.array([_x.min(),_x.max()])
    c_y = slope*c_x + intercept
    plt.plot(c_x,c_y,'r--',label='Regression line')
    
    # plot confidence limits
    plt.plot(p_x,lower,':',c='grey',label='Lower confidence limit (95%)')
    plt.plot(p_x,upper,':',c='grey',label='Upper confidence limit (95%)')

In [ ]:
SMALL_SIZE = 16
MEDIUM_SIZE = 16
BIGGER_SIZE = 18

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title


---
## Make dataset

First I need to make a combined dataset to make these investigations easier

OPTIONAL: Read in the ground based data timeseries for Gaylor Pit and CUES sites

In [ ]:
## Gaylor Pit
#tuol_df = pd.read_pickle('data/goes-tuolumne_2017-2020.pkl')
#tuol_df['timeUTC'] = tuol_df.index + pd.Timedelta(hours=8)
#tuol_df.set_index('timeUTC',inplace=True)
#tuol_ds = tuol_df.to_xarray().squeeze()
#
## CUES
#cues_df = pd.read_pickle('data/goes-cues_2017-2020.pkl')
#cues_df['timeUTC'] = cues_df.index + pd.Timedelta(hours=8)
#cues_df.set_index('timeUTC',inplace=True)
#cues_ds = cues_df.to_xarray().squeeze()

Set up working area geometry

In [ ]:
# Larger area around Tuolumne further south and east to include Mammoth
(utm_e_UL, utm_n_UL) = 282500, 4205000
(utm_e_UR, utm_n_UR) = 305000, 4205000
(utm_e_LR, utm_n_LR) = 327500, 4160000
(utm_e_LL, utm_n_LL) = 305000, 4160000

geometry = [
    {
        'type': 'Polygon',
        'coordinates': [[
            [utm_e_UL, utm_n_UL],
            [utm_e_UR, utm_n_UR],
            [utm_e_LR, utm_n_LR],
            [utm_e_LL, utm_n_LL]
        ]]
    }
]

Find datasets (orthorectified GOES ABI or original GOES ABI)

In [ ]:
#filepaths = glob.glob('/storage/spestana/output_ASTER-MODIS-GOES/*_orthorectified.nc')
##filepaths = glob.glob('/storage/spestana/output_ASTER-MODIS-GOES/*_original.nc')

Open all datasets

In [ ]:
#ds = xr.open_mfdataset(filepaths, concat_dim='time', combine='nested')
######ds = ds.rio.clip(geometry)

Open my DEM, reproject to UTM, add to the dataset

In [ ]:
#dem = xr.open_rasterio('dem/tuolumne_cues_dem.tif').rio.reproject_match(ds)
#ds['dem'] = dem.squeeze()

OLD, skip this: Compute slope and aspect and add to the dataset

In [ ]:
#dem_array = rd.rdarray(ds.dem.values, no_data=np.nan).squeeze()
#slope = rd.TerrainAttribute(dem_array, attrib='slope_riserun')
#ds['slope'] = (['y','x'], slope)
#aspect = rd.TerrainAttribute(dem_array, attrib='aspect')
#ds['aspect'] = (['y','x'], aspect)
## aspect looks like it puts 0=360 at East not north. 

---

Create fractional vegetated area from TCC via a binary vegetation map

In [ ]:
#def fveg_mapZonalStats(zones, zonalstats, stat_name):
#    ''' Function for mapping the zonal statistics back to the original grid to get a 2D map of the chosen statistic'''
#    # create an empty array for this summary stat
#    zonal_stat = np.zeros_like(zones.values, dtype=np.float64)
#
#    # for each zone
#    for zone_n in zonalstats.index.values:
#        # get the summary stat for that zone, 
#        # and assign it to the correct locations in the zonal_stat array
#        #try:
#        zonal_stat[zones.values==zone_n] = zonalstats['{}'.format(stat_name)].loc[zone_n]
#        #except: #MaskError: Cannot convert masked element to a Python int.
#        #    zonal_stat[zones.values==zone_n] = -9999
#
#    # convert this to an xarray data array with the proper name
#    zonal_stat_da = xr.DataArray(zonal_stat, 
#                                 dims=["y", "x"],
#                                 coords=dict(
#                                             x=(["x"], zones.x),
#                                             y=(["y"], zones.y),
#                                             ),
#                                 name='zonal_{}'.format(stat_name))
#    # remove nodata values
#    zonal_stat_da = zonal_stat_da.where(zonal_stat_da!=-9999, np.nan)
#
#    return zonal_stat_da     

Open TCC and add to the dataset

In [ ]:
#tcc = xr.open_rasterio('data/NLCD_2016_Tree_Canopy_L48_20190831_2xOlzdukUemPqURl8ckP.tiff').rio.reproject_match(ds)
#tcc = tcc.where(tcc!=tcc._FillValue)
#ds['tcc'] = tcc.squeeze()
#

New version: making fveg from binary tcc

In [ ]:
#def tcc_to_binary_forest(tcc, threshold):
#    
#    intermediate_map = tcc.where((np.isnan(tcc))|(tcc>threshold),0)
#    
#    binary_forest = intermediate_map.where((np.isnan(intermediate_map))|(intermediate_map<threshold), 1)
#    
#    return binary_forest
#
#ds['binary_tcc_gt10'] = tcc_to_binary_forest(tcc, 10).squeeze()
#ds['binary_tcc_gt20'] = tcc_to_binary_forest(tcc, 20).squeeze()
#ds['binary_tcc_gt30'] = tcc_to_binary_forest(tcc, 30).squeeze()
#ds['binary_tcc_gt40'] = tcc_to_binary_forest(tcc, 40).squeeze()
#ds['binary_tcc_gt50'] = tcc_to_binary_forest(tcc, 50).squeeze()

Plot an image of this if we want to inspect what our choice of threshold did

In [ ]:
#fig, axs = plt.subplots(nrows=2, ncols=3, figsize=(40,20))
#[ax1, ax2, ax3, ax4, ax5, ax6] = axs.ravel()
#
#ds.binary_tcc_gt10.where(ds.binary_tcc_gt10!=0).plot(cmap='spring',alpha=1, ax=ax1, add_colorbar=False)
#ds.binary_tcc_gt20.where(ds.binary_tcc_gt20!=0).plot(cmap='spring',alpha=1, ax=ax2, add_colorbar=False)
#ds.binary_tcc_gt30.where(ds.binary_tcc_gt30!=0).plot(cmap='spring',alpha=1, ax=ax3, add_colorbar=False)
#ds.binary_tcc_gt40.where(ds.binary_tcc_gt40!=0).plot(cmap='spring',alpha=1, ax=ax4, add_colorbar=False)
#ds.binary_tcc_gt50.where(ds.binary_tcc_gt50!=0).plot(cmap='spring',alpha=1, ax=ax5, add_colorbar=False)
#
#for ax in  [ax1, ax2, ax3, ax4, ax5]:
#    tcc.plot(ax=ax, cmap='Greys_r', add_colorbar=False, zorder=-99)
#    ax.set_ylim((4.18e6,4.20e6))
#    ax.set_xlim((290000,310000))

I'm going to base my binary forest map on a threshold of 30% TCC

In [ ]:
#da_fveg_mean_stack = []
#for i in range(0,len(ds.time)):
#    fveg_mean = ds.isel(time=i).groupby('ast_goes_goes_zones').mean().binary_tcc_gt30.to_dataframe()
#    da_fveg_mean = fveg_mapZonalStats(ds.isel(time=i).ast_goes_goes_zones, fveg_mean, 'binary_tcc_gt30')
#    da_fveg_mean_stack.append(da_fveg_mean)
#    
#ds_fveg_mean = xr.concat(da_fveg_mean_stack, 'time' )
#ds['fveg'] = ds_fveg_mean.where(~np.isnan(ds.ast_goes_goes_zones))

In [ ]:
### Old version: Making fveg from tcc mean

###fveg = (tcc - tcc.min()) / (tcc.max() - tcc.min()) # normalizing tcc to get fveg
###ds['fveg'] = fveg.squeeze()
###
###da_fveg_mean_stack = []
###for i in range(0,len(ds.time)):
###    fveg_mean = ds.isel(time=i).groupby('ast_goes_goes_zones').mean().fveg.drop(['spatial_ref','band','time']).to_dataframe()
###    da_fveg_mean = fveg_mapZonalStats(ds.isel(time=i).ast_goes_goes_zones, fveg_mean, 'fveg')
###    da_fveg_mean_stack.append(da_fveg_mean)
###    
###ds_fveg_mean = xr.concat(da_fveg_mean_stack, 'time' )
###ds['zonal_mean_fveg'] = ds_fveg_mean


Save out datafile to a single netcdf

In [ ]:
#ds.to_netcdf('/storage/spestana/ASTER-MODIS-GOES-DEM-FVEG_v3_orthorectified.nc')

---
---
## Open dataset (if already created above)

In [ ]:
ds = xr.open_dataset('/storage/spestana/ASTER-MODIS-GOES-DEM-FVEG_v3_orthorectified.nc')
ds = ds.rio.clip(geometry)

In [ ]:
ds

---
---

### Elevation

deltaT versus Elevation for all 27 observations

In [ ]:
conf = 0.975

y = np.array([ds.dem.values] * 27).ravel()

# MODIS-ASTER deltaT
plt.figure(figsize=(10,5))
x = ds.ast_mod_mean_diff_tb.values.ravel()
plt.scatter(x,y,marker='.',c='k',alpha=0.02,s=1)
_x = x[~np.isnan(x)]
_y = y[~np.isnan(x)]
slope, intercept, r, p, se = stats.linregress(_x, _y)
print(slope, intercept, r, p, se)
plt.plot(np.linspace(_x.min(),_x.max()),
         slope*np.linspace(_x.min(),_x.max())+intercept,
         '--r',zorder=99)
plt.ylabel('Elevation [m]')
plt.xlabel('MODIS-ASTER $\Delta T\,[\degree C]$')
plt.title('MODIS-ASTER $\Delta T\,[\degree C]$\nversus Elevation [m]')
plot_regression_confidence_intervals(_x,_y,conf)

# ABI-ASTER deltaT
plt.figure(figsize=(10,5))
x = ds.ast_goes_mean_diff_tb.values.ravel()
plt.scatter(x,y,marker='.',c='k',alpha=0.02,s=1)
_x = x[~np.isnan(x)]
_y = y[~np.isnan(x)]
slope, intercept, r, p, se = stats.linregress(_x, _y)
print(slope, intercept, r, p, se)
plt.plot(np.linspace(_x.min(),_x.max()),
         slope*np.linspace(_x.min(),_x.max())+intercept,
         '--r',zorder=99)
plt.ylabel('Elevation [m]')
plt.xlabel('ABI-ASTER $\Delta T\,[\degree C]$')
plt.title('ABI-ASTER $\Delta T\,[\degree C]$\nversus Elevation [m]')
plot_regression_confidence_intervals(_x,_y,conf)

# Absolute value of ABI-ASTER deltaT
plt.figure(figsize=(10,5))
x = np.abs(ds.ast_goes_mean_diff_tb.values.ravel())
plt.scatter(x,y,marker='.',c='k',alpha=0.02,s=1)
_x = x[~np.isnan(x)]
_y = y[~np.isnan(x)]
slope, intercept, r, p, se = stats.linregress(_x, _y)
print(slope, intercept, r, p, se)
plt.plot(np.linspace(_x.min(),_x.max()),
         slope*np.linspace(_x.min(),_x.max())+intercept,
         '--r',zorder=99)
plt.ylabel('Elevation [m]')
plt.xlabel('|ABI-ASTER $\Delta T$| $[\degree C]$')
plt.title('|ABI-ASTER $\Delta T$| $[\degree C]$\nversus Elevation [m]')
plot_regression_confidence_intervals(_x,_y,conf)

mean deltaT versus Elevation

In [ ]:
conf = 0.975

y = ds.dem.values.ravel()

# MODIS-ASTER deltaT
plt.figure(figsize=(5,5))
x = ds.ast_mod_mean_diff_tb.mean(axis=0).values.ravel()
plt.scatter(x,y,marker='.',c='k',alpha=0.02,s=5)
_x = x[~np.isnan(x)]
_y = y[~np.isnan(x)]
slope, intercept, r, p, se = stats.linregress(_x, _y)
print(slope, intercept, r, p, se)
plt.plot(np.linspace(_x.min(),_x.max()),
         slope*np.linspace(_x.min(),_x.max())+intercept,
         '--r',zorder=99)
plt.ylabel('Elevation [m]')
plt.xlabel('MODIS-ASTER $\Delta T\,[\degree C]$')
plt.title('MODIS-ASTER $\Delta T\,[\degree C]$\nversus Elevation [m]')
plot_regression_confidence_intervals(_x,_y,conf)

# ABI-ASTER deltaT
plt.figure(figsize=(5,5))
x = ds.ast_goes_mean_diff_tb.mean(axis=0).values.ravel()
plt.scatter(x,y,marker='.',c='k',alpha=0.02,s=5)
_x = x[~np.isnan(x)]
_y = y[~np.isnan(x)]
slope, intercept, r, p, se = stats.linregress(_x, _y)
print(slope, intercept, r, p, se)
#plt.plot(np.linspace(_x.min(),_x.max()),
#         slope*np.linspace(_x.min(),_x.max())+intercept,
#         '--r',zorder=99)
plt.ylabel('Elevation [m]')
plt.xlabel('ABI-ASTER $\Delta T\,[\degree C]$')
plt.title('ABI-ASTER $\Delta T\,[\degree C]$\nversus Elevation [m]')
#plot_regression_confidence_intervals(_x,_y,conf)

# Absolute value of ABI-ASTER deltaT
plt.figure(figsize=(5,5))
x = np.abs(ds.ast_goes_mean_diff_tb.mean(axis=0).values.ravel())
plt.scatter(x,y,marker='.',c='k',alpha=0.02,s=5)
_x = x[~np.isnan(x)]
_y = y[~np.isnan(x)]
slope, intercept, r, p, se = stats.linregress(_x, _y)
print(slope, intercept, r, p, se)
#plt.plot(np.linspace(_x.min(),_x.max()),
#         slope*np.linspace(_x.min(),_x.max())+intercept,
#         '--r',zorder=99)
plt.ylabel('Elevation [m]')
plt.xlabel('|ABI-ASTER $\Delta T$| $[\degree C]$')
plt.title('|ABI-ASTER $\Delta T$| $[\degree C]$\nversus Elevation [m]')
#plot_regression_confidence_intervals(_x,_y,conf)

---
---
---

### download another dem to compute slope/aspect/hillshade/etd with

In [ ]:
get_dem(demtype='SRTMGL3', 
        bounds=(-120, 36 , -118, 40), 
        out_fn='dem_4.tif')

In [ ]:
dem_elevation = rxr.open_rasterio('dem_4.tif')
dem_elevation = dem_elevation.rio.reproject_match(ds.dem)

In [ ]:
!gdaldem aspect dem_4.tif aspect_4.tif

In [ ]:
fig, ax = plt.subplots(figsize=(6,5))
dem_elevation.plot(ax=ax, vmin=1300, vmax=4000, cmap='terrain')
ax.axis('off')
ax.set_title('')
ax.plot([utm_e_UL, utm_e_UR, utm_e_LR, utm_e_LL, utm_e_UL],
                 [utm_n_UL, utm_n_UR, utm_n_LR, utm_n_LL, utm_n_UL],
                 linestyle='--', color='w', lw=3)

Compute Aspect Map

In [ ]:
dem_aspect = rxr.open_rasterio('aspect_4.tif')
dem_aspect = dem_aspect.rio.reproject_match(ds.dem)
dem_aspect = dem_aspect.where(dem_aspect != -9999)

In [ ]:
fig, ax = plt.subplots(figsize=(6,5))
dem_aspect.plot(ax=ax, cmap='hsv')
ax.axis('off')
ax.set_title('')
ax.plot([utm_e_UL, utm_e_UR, utm_e_LR, utm_e_LL, utm_e_UL],
                 [utm_n_UL, utm_n_UR, utm_n_LR, utm_n_LL, utm_n_UL],
                 linestyle='-', color='k', lw=3)

Compute Slope Map (need to reproject first into UTM so we compute vertical meters / horizontal meters)

In [ ]:
!gdalwarp -t_srs EPSG:32611 dem_4.tif dem_4utm.tif

In [ ]:
!gdaldem slope dem_4utm.tif slope_4.tif

In [ ]:
dem_slope = rxr.open_rasterio('slope_4.tif')
dem_slope = dem_slope.rio.reproject_match(ds.dem)
dem_slope = dem_slope.where(dem_slope != -9999.) 
#dem_slope = dem_slope.where(dem_slope != -9999)
#dem_slope = dem_slope.where(dem_slope != 0)

In [ ]:
fig, ax = plt.subplots(figsize=(6,5))
dem_slope.plot(ax=ax, cmap='Greys_r')
ax.axis('off')
ax.set_title('')
ax.plot([utm_e_UL, utm_e_UR, utm_e_LR, utm_e_LL, utm_e_UL],
                 [utm_n_UL, utm_n_UR, utm_n_LR, utm_n_LL, utm_n_UL],
                 linestyle='--', color='w', lw=3)

Function for computing GOES Local Zenith Angles, and Azimuth

In [ ]:
def goes_lza(lat_ssp, lon_ssp, lat, lon, H=42164.16, r_eq=6378.137):
    
    '''
    Compute the Locan Zenith Angle for a point on Earth surface to a GOES-R geostationary satellite.
        See more details from NOAA here: 
        https://www.ncdc.noaa.gov/sites/default/files/attachments/GOES-R_ABI_local_zenith_angle_description.docx
    
    Inputs:
        GOES-R satellite position
            lat_ssp: sub-satellite point latitude [degrees]
            lon_ssp: sub-satellite point longitude [degrees]
    
        View point (on Earth's surface) position
            lat: view point latitude on Earth's surfaace [degrees]
            lon: view point longitude on Earth's surface [degrees]
            elev: view point elevation (heigh above GRS80 ellispoid) [km]
            
        Earth model parameters (optional)
            H: satellite distance to Earth center [km] (defaults to 42164.16 km)
            r_eq: Earth semi-major axis (GRS80 ellipsoid) [km] (defaults to 6378.137 km)
            
    Returns:
        LZA: local zenith angle [degrees]
        is_point_visible: True/False flag indicating if the ground point is actually visible to the satellite
    
    '''

    # intermediate calculation
    B = np.arccos( np.cos(np.radians(lat)-np.radians(lat_ssp)) * np.cos(np.radians(lon)-np.radians(lon_ssp)) )

    # determine if point is visible to the satellite
    is_point_visible = (B < np.arccos(r_eq / (H+r_eq)))

    # compute LZA
    LZA_radians = np.arcsin( (H * np.sin(B) ) / ( np.sqrt( H**2 + r_eq**2 - 2*H*r_eq*np.cos(B) ) ) )
    
    # convert LZA from radians to degrees
    LZA = LZA_radians * 180/np.pi
    
    return LZA, is_point_visible

def goes_azi(lat_ssp, lon_ssp, lat, lon):
    
    '''quick calculation of azimuth for geostationary satellite, not GOES specific, spherical Earth assumption
    http://tiij.org/issues/issues/3_2/3_2e.html'''
    
    azi = 180 + np.degrees( np.arctan(np.tan(np.radians(lon_ssp - lon))/np.sin(np.radians(lat))) )
    
    return azi.T

Compute LZA and Azimuth maps

In [ ]:
# when t is 0,1,2,3,4... ssp_lon = -89.5

In [ ]:
dem_latlon = rxr.open_rasterio('dem_4.tif') # need to open the dem in its WGS84 lat/lon projection
lza_map, visible = goes_lza(0, -75.2, dem_latlon.y, dem_latlon.x)
lza_map = lza_map.rio.reproject_match(ds.dem)
azimuth_map = goes_azi(0, 75.2, dem_latlon.y, -dem_latlon.x)
azimuth_map = azimuth_map.rio.reproject_match(ds.dem)


fig, ax = plt.subplots(figsize=(6,5))
lza_map.plot(ax=ax)
ax.axis('off')
ax.set_title(f'mean LZA = {np.round(np.mean(lza_map).values,2)}')
ax.plot([utm_e_UL, utm_e_UR, utm_e_LR, utm_e_LL, utm_e_UL],
                 [utm_n_UL, utm_n_UR, utm_n_LR, utm_n_LL, utm_n_UL],
                 linestyle='--', color='w', lw=3)

fig, ax = plt.subplots(figsize=(6,5))
azimuth_map.plot(ax=ax)
ax.axis('off')
ax.set_title(f'mean AZI = {np.round(np.mean(azimuth_map).values,2)}')
ax.plot([utm_e_UL, utm_e_UR, utm_e_LR, utm_e_LL, utm_e_UL],
                 [utm_n_UL, utm_n_UR, utm_n_LR, utm_n_LL, utm_n_UL],
                 linestyle='--', color='w', lw=3)

Hillshade function

In [ ]:
def hillshade(slope, aspect, zenith, azimuth):
    slope_rad = np.radians(slope)
    aspect_rad = np.radians(aspect)
    zenith_rad = np.radians(zenith)
    azimuth_rad = np.radians(azimuth)
    hillshade = np.cos(zenith_rad)*np.cos(slope_rad) + np.sin(zenith_rad)*np.sin(slope_rad)*np.cos(azimuth_rad - aspect_rad)
    return hillshade

### Test hillshade function
#_slopes = np.array([np.arange(0,90,1)]*360).T
#_aspects = np.array([np.arange(0,360,1)]*90)
#
#test_hs = hillshade(_slopes, _aspects, 45, 90)
#
#plt.pcolormesh(test_hs,vmin=-1,vmax=1,cmap='RdBu_r')
#plt.colorbar(label='hillshade')
#plt.ylabel('slope')
#plt.xlabel('aspect')

### Compute two hillshades
* **hs_sat** for the satellite's view
* **hs_sun** for solar illumination

In [ ]:
hs_sat = hillshade(dem_slope, dem_aspect, lza_map, azimuth_map)
hs_sat = hs_sat.rio.clip(geometry)

In [ ]:
fig, ax = plt.subplots(figsize=(6,5))
hs_sat.plot(cmap='Greys_r',vmin=0,vmax=1,ax=ax,alpha=1)
ax.axis('off')
ax.set_title('')
ax.plot([utm_e_UL, utm_e_UR, utm_e_LR, utm_e_LL, utm_e_UL],
                 [utm_n_UL, utm_n_UR, utm_n_LR, utm_n_LL, utm_n_UL],
                 linestyle='-', color='k', lw=3);

fig, ax = plt.subplots(figsize=(6,5))
hs_sat.plot(cmap='Greys_r',vmin=0,vmax=1,ax=ax,alpha=1)
hs_sat.where(hs_sat<=0).plot(cmap='spring',vmin=0,vmax=1,ax=ax,alpha=1, add_colorbar=False)
ax.axis('off')
ax.set_title('')
ax.plot([utm_e_UL, utm_e_UR, utm_e_LR, utm_e_LL, utm_e_UL],
                 [utm_n_UL, utm_n_UR, utm_n_LR, utm_n_LL, utm_n_UL],
                 linestyle='-', color='k', lw=3);

---

### Sunlight hillshade maps
* use pysolar to get sun positions for each DEM gridcell for each of the 27 date/times of GOES observations

In [ ]:
#date = datetime.datetime(2007, 2, 18, 18, 59, 59, tzinfo=datetime.timezone.utc)
dates = [pd.Timestamp( this_date, tzinfo=datetime.timezone.utc ) for this_date in ds.time.values]
date = dates[1]

longitude = dem_latlon.x
latitude = dem_latlon.y

In [ ]:
solar_altitude_deg = pysolar.solar.get_altitude(latitude, longitude, date).transpose('y', 'x')
solar_altitude_deg = solar_altitude_deg.rio.reproject_match(ds.dem)
solar_lza_deg = 90 - solar_altitude_deg
#.rio.clip(geometry)

solar_azimuth_deg = pysolar.solar.get_azimuth(latitude, longitude, date).transpose('y', 'x')
solar_azimuth_deg = azimuth_deg.rio.reproject_match(ds.dem)
#.rio.clip(geometry)

In [ ]:
fig, ax = plt.subplots(figsize=(6,5))
solar_lza_deg.plot(ax=ax,alpha=1)
ax.axis('off')
ax.set_title(f'Solar LZA {date}')
ax.plot([utm_e_UL, utm_e_UR, utm_e_LR, utm_e_LL, utm_e_UL],
                 [utm_n_UL, utm_n_UR, utm_n_LR, utm_n_LL, utm_n_UL],
                 linestyle='--', color='w', lw=3);

fig, ax = plt.subplots(figsize=(6,5))
solar_azimuth_deg.plot(ax=ax,alpha=1)
ax.axis('off')
ax.set_title(f'Solar Azimuth {date}')
ax.plot([utm_e_UL, utm_e_UR, utm_e_LR, utm_e_LL, utm_e_UL],
                 [utm_n_UL, utm_n_UR, utm_n_LR, utm_n_LL, utm_n_UL],
                 linestyle='--', color='w', lw=3);

Compute solar hillshade for date/time

In [ ]:
hs_sun = hillshade(dem_slope, dem_aspect, solar_lza_deg, solar_azimuth_deg)
hs_sun = hs_sun.rio.clip(geometry)

In [ ]:
fig, ax = plt.subplots(figsize=(6,5))
hs_sun.plot(cmap='Greys_r',vmin=0,vmax=1,ax=ax,alpha=1)
hs_sun.where(hs_sun<=0).plot(cmap='spring',vmin=0,vmax=1,ax=ax,alpha=1, add_colorbar=False)
ax.axis('off')
ax.set_title('')
ax.plot([utm_e_UL, utm_e_UR, utm_e_LR, utm_e_LL, utm_e_UL],
                 [utm_n_UL, utm_n_UR, utm_n_LR, utm_n_LL, utm_n_UL],
                 linestyle='-', color='k', lw=3);

Make a Diurnal Anisotropic Heating index 
* DAH from (Böhner & Antonic, 2009)
* $\alpha_{max} = 202.5^o (SSW)$ from Cristea et al., 2017 in Tuolumne area

In [ ]:
def diurnal_anisotropic_heat(aspect, slope, a_max):
    DAH = np.cos(np.radians(a_max - aspect)) * np.arctan(np.radians(slope))
    return DAH

In [ ]:
DAH = diurnal_anisotropic_heat(dem_aspect, dem_slope, solar_azimuth_deg)

In [ ]:
fig, ax = plt.subplots(figsize=(6,5))
DAH.plot(ax=ax,alpha=1)
ax.axis('off')
ax.set_title(f'DAH')
ax.plot([utm_e_UL, utm_e_UR, utm_e_LR, utm_e_LL, utm_e_UL],
                 [utm_n_UL, utm_n_UR, utm_n_LR, utm_n_LL, utm_n_UL],
                 linestyle='-', color='k', lw=3);

---

### "Hillshade"

In [ ]:
conf = 0.975

y = np.array([hs.values] * 27).ravel()

# MODIS-ASTER deltaT
plt.figure(figsize=(5,5))
x = ds.ast_mod_mean_diff_tb.values.ravel()
plt.scatter(x,y,marker='.',c='k',alpha=0.02)
_x = x[~np.isnan(x)]
_y = y[~np.isnan(x)]
slope, intercept, r, p, se = stats.linregress(_x, _y)
plt.plot(np.linspace(_x.min(),_x.max()),
         slope*np.linspace(_x.min(),_x.max())+intercept,
         '--r',zorder=99)
plt.ylabel('hs_sat')
plt.xlabel('MODIS-ASTER $\Delta T\,[\degree C]$')
plt.title('MODIS-ASTER $\Delta T\,[\degree C]$\nversus hs_sat')
plot_regression_confidence_intervals(_x,_y,conf)

# ABI-ASTER deltaT
plt.figure(figsize=(5,5))
x = ds.ast_goes_mean_diff_tb.values.ravel()
plt.scatter(x,y,marker='.',c='k',alpha=0.02)
_x = x[~np.isnan(x)]
_y = y[~np.isnan(x)]
slope, intercept, r, p, se = stats.linregress(_x, _y)
plt.plot(np.linspace(_x.min(),_x.max()),
         slope*np.linspace(_x.min(),_x.max())+intercept,
         '--r',zorder=99)
plt.ylabel('hs_sat')
plt.xlabel('ABI-ASTER $\Delta T\,[\degree C]$')
plt.title('ABI-ASTER $\Delta T\,[\degree C]$\nversus hs_sat')
plot_regression_confidence_intervals(_x,_y,conf)

# Absolute value of ABI-ASTER deltaT
plt.figure(figsize=(5,5))
x = np.abs(ds.ast_goes_mean_diff_tb.values.ravel())
plt.scatter(x,y,marker='.',c='k',alpha=0.02)
_x = x[~np.isnan(x)]
_y = y[~np.isnan(x)]
slope, intercept, r, p, se = stats.linregress(_x, _y)
plt.plot(np.linspace(_x.min(),_x.max()),
         slope*np.linspace(_x.min(),_x.max())+intercept,
         '--r',zorder=99)
plt.ylabel('hs_sat')
plt.xlabel('|ABI-ASTER $\Delta T$| $[\degree C]$')
plt.title('|ABI-ASTER $\Delta T$| $[\degree C]$\nversus hs_sat')
plot_regression_confidence_intervals(_x,_y,conf)

In [ ]:
y = hs.values.ravel()

# MODIS-ASTER deltaT
plt.figure(figsize=(5,5))
x = ds.ast_mod_mean_diff_tb.mean(axis=0).values.ravel()
plt.scatter(x,y,marker='.',c='k',alpha=0.02,s=5)
_x = x[~np.isnan(x)]
_y = y[~np.isnan(x)]
slope, intercept, r, p, se = stats.linregress(_x, _y)
plt.plot(np.linspace(_x.min(),_x.max()),
         slope*np.linspace(_x.min(),_x.max())+intercept,
         '--r',zorder=99)
plt.ylabel('hs_sat')
plt.xlabel('MODIS-ASTER $\Delta T\,[\degree C]$')
plt.title('MODIS-ASTER $\Delta T\,[\degree C]$\nversus hs_sat')
plot_regression_confidence_intervals(_x,_y,conf)

# ABI-ASTER deltaT
plt.figure(figsize=(5,5))
x = ds.ast_goes_mean_diff_tb.mean(axis=0).values.ravel()
plt.scatter(x,y,marker='.',c='k',alpha=0.02,s=5)
_x = x[~np.isnan(x)]
_y = y[~np.isnan(x)]
slope, intercept, r, p, se = stats.linregress(_x, _y)
plt.plot(np.linspace(_x.min(),_x.max()),
         slope*np.linspace(_x.min(),_x.max())+intercept,
         '--r',zorder=99)
plt.ylabel('hs_sat')
plt.xlabel('ABI-ASTER $\Delta T\,[\degree C]$')
plt.title('ABI-ASTER $\Delta T\,[\degree C]$\nversus hs_sat')
plot_regression_confidence_intervals(_x,_y,conf)

# Absolute value of ABI-ASTER deltaT
plt.figure(figsize=(5,5))
x = np.abs(ds.ast_goes_mean_diff_tb.mean(axis=0).values.ravel())
plt.scatter(x,y,marker='.',c='k',alpha=0.02,s=5)
_x = x[~np.isnan(x)]
_y = y[~np.isnan(x)]
slope, intercept, r, p, se = stats.linregress(_x, _y)
plt.plot(np.linspace(_x.min(),_x.max()),
         slope*np.linspace(_x.min(),_x.max())+intercept,
         '--r',zorder=99)
plt.ylabel('hs_sat')
plt.xlabel('|ABI-ASTER $\Delta T$| $[\degree C]$')
plt.title('|ABI-ASTER $\Delta T$| $[\degree C]$\nversus hs_sat')
plot_regression_confidence_intervals(_x,_y,conf)

---

### Slope

In [ ]:
conf = 0.975

y = np.array([dem_slope.values] * 27).ravel()

# MODIS-ASTER deltaT
plt.figure(figsize=(5,5))
x = ds.ast_mod_mean_diff_tb.values.ravel()
plt.scatter(x,y,marker='.',c='k',alpha=0.02)
_x = x[~np.isnan(x)]
_y = y[~np.isnan(x)]
slope, intercept, r, p, se = stats.linregress(_x, _y)
plt.plot(np.linspace(_x.min(),_x.max()),
         slope*np.linspace(_x.min(),_x.max())+intercept,
         '--r',zorder=99)
plt.ylabel('Slope [m/m]')
plt.xlabel('MODIS-ASTER $\Delta T\,[\degree C]$')
plt.title('MODIS-ASTER $\Delta T\,[\degree C]$\nversus Slope')
plot_regression_confidence_intervals(_x,_y,conf)

# ABI-ASTER deltaT
plt.figure(figsize=(5,5))
x = ds.ast_goes_mean_diff_tb.values.ravel()
plt.scatter(x,y,marker='.',c='k',alpha=0.02)
_x = x[~np.isnan(x)]
_y = y[~np.isnan(x)]
slope, intercept, r, p, se = stats.linregress(_x, _y)
plt.plot(np.linspace(_x.min(),_x.max()),
         slope*np.linspace(_x.min(),_x.max())+intercept,
         '--r',zorder=99)
plt.ylabel('Slope [m/m]')
plt.xlabel('ABI-ASTER $\Delta T\,[\degree C]$')
plt.title('ABI-ASTER $\Delta T\,[\degree C]$\nversus Slope')
plot_regression_confidence_intervals(_x,_y,conf)

# Absolute value of ABI-ASTER deltaT
plt.figure(figsize=(5,5))
x = np.abs(ds.ast_goes_mean_diff_tb.values.ravel())
plt.scatter(x,y,marker='.',c='k',alpha=0.02)
_x = x[~np.isnan(x)]
_y = y[~np.isnan(x)]
slope, intercept, r, p, se = stats.linregress(_x, _y)
plt.plot(np.linspace(_x.min(),_x.max()),
         slope*np.linspace(_x.min(),_x.max())+intercept,
         '--r',zorder=99)
plt.ylabel('Slope [m/m]')
plt.xlabel('|ABI-ASTER $\Delta T$| $[\degree C]$')
plt.title('|ABI-ASTER $\Delta T$| $[\degree C]$\nversus Slope')
plot_regression_confidence_intervals(_x,_y,conf)

In [ ]:
y = dem_slope.values.ravel()

# MODIS-ASTER deltaT
plt.figure(figsize=(5,5))
x = ds.ast_mod_mean_diff_tb.mean(axis=0).values.ravel()
plt.scatter(x,y,marker='.',c='k',alpha=0.02,s=5)
_x = x[~np.isnan(x)]
_y = y[~np.isnan(x)]
slope, intercept, r, p, se = stats.linregress(_x, _y)
plt.plot(np.linspace(_x.min(),_x.max()),
         slope*np.linspace(_x.min(),_x.max())+intercept,
         '--r',zorder=99)
plt.ylabel('Slope [m/m]')
plt.xlabel('MODIS-ASTER $\Delta T\,[\degree C]$')
plt.title('MODIS-ASTER $\Delta T\,[\degree C]$\nversus Slope')
plot_regression_confidence_intervals(_x,_y,conf)

# ABI-ASTER deltaT
plt.figure(figsize=(5,5))
x = ds.ast_goes_mean_diff_tb.mean(axis=0).values.ravel()
plt.scatter(x,y,marker='.',c='k',alpha=0.02,s=5)
_x = x[~np.isnan(x)]
_y = y[~np.isnan(x)]
slope, intercept, r, p, se = stats.linregress(_x, _y)
plt.plot(np.linspace(_x.min(),_x.max()),
         slope*np.linspace(_x.min(),_x.max())+intercept,
         '--r',zorder=99)
plt.ylabel('Slope [m/m]')
plt.xlabel('ABI-ASTER $\Delta T\,[\degree C]$')
plt.title('ABI-ASTER $\Delta T\,[\degree C]$\nversus Slope')
plot_regression_confidence_intervals(_x,_y,conf)

# Absolute value of ABI-ASTER deltaT
plt.figure(figsize=(5,5))
x = np.abs(ds.ast_goes_mean_diff_tb.mean(axis=0).values.ravel())
plt.scatter(x,y,marker='.',c='k',alpha=0.02,s=5)
_x = x[~np.isnan(x)]
_y = y[~np.isnan(x)]
slope, intercept, r, p, se = stats.linregress(_x, _y)
plt.plot(np.linspace(_x.min(),_x.max()),
         slope*np.linspace(_x.min(),_x.max())+intercept,
         '--r',zorder=99)
plt.ylabel('Slope [m/m]')
plt.xlabel('|ABI-ASTER $\Delta T$| $[\degree C]$')
plt.title('|ABI-ASTER $\Delta T$| $[\degree C]$\nversus Slope')
plot_regression_confidence_intervals(_x,_y,conf)

### Aspect

In [ ]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

xval = np.arange(0, 2*np.pi, 0.01)
yval = np.ones_like(xval)

colormap = plt.get_cmap('hsv')
norm = mpl.colors.Normalize(0.0, 2*np.pi)

ax = plt.subplot(1,1,1, polar=True)
ax.scatter(xval, yval, c=xval, s=1000, cmap=colormap, norm=norm, linewidths=0)
#ax.set_yticks([])
ax.axis('off')

In [ ]:
fig, ax = plt.subplots(figsize=(6,5))
dem_aspect.plot(ax=ax, cmap='hsv')
ax.axis('off')
ax.set_title('')
ax.plot([utm_e_UL, utm_e_UR, utm_e_LR, utm_e_LL, utm_e_UL],
                 [utm_n_UL, utm_n_UR, utm_n_LR, utm_n_LL, utm_n_UL],
                 linestyle='-', color='k', lw=3)

In [ ]:
plt.hist(dem_aspect.values.ravel());

In [ ]:
ds_day = ds.where(ds.time.dt.hour==18)
ds_night = ds.where(ds.time.dt.hour==6)

In [ ]:
def pol2cart(rho, phi, rotate):
    # given a point's polar coordinates rho (radial distance), phi (clockwise azimuth angle), 
    # and rotate (angle to add to phi to put 0 degrees at the top like a compass)
    #https://stackoverflow.com/questions/20924085/python-conversion-between-coordinates
    x = rho * np.cos(np.radians(-phi+rotate))
    y = rho * np.sin(np.radians(-phi+rotate))
    return(x, y)

In [ ]:
ds_day = ds.where(ds.time.dt.hour==18, drop=True)
ds_night = ds.where(ds.time.dt.hour==6, drop=True)

In [ ]:
azis_all = np.array([dem_aspect.values]*len(ds.time)).ravel() # azimuth angles these are oriented with 0 as south, 180 as north
azis_day = np.array([dem_aspect.values]*len(ds_day.time)).ravel() # azimuth angles these are oriented with 0 as south, 180 as north
azis_night = np.array([dem_aspect.values]*len(ds_night.time)).ravel() # azimuth angles these are oriented with 0 as south, 180 as north
temps_day = ds_day.ast_goes_mean_diff_tb.values.ravel() # ABI temperatures day
temps_night = ds_night.ast_goes_mean_diff_tb.values.ravel() # ABI temperatures night
#temps_day = ds_day.ast_mod_mean_diff_tb.values.ravel() # MODIS temperatures day
#temps_night = ds_night.ast_mod_mean_diff_tb.values.ravel() # MODIS temperatures night

rotate = 90 # rotate 90 for 0 at top, rotate -90 for 0 at bottom 

# for all data
x_day,y_day = pol2cart(temps_day,azis_day,rotate)
x_night,y_night = pol2cart(temps_night,azis_night,rotate) 

# average by 1 degree bins
azis_day_1deg = np.round(azis_day,0)
azis_night_1deg = np.round(azis_night,0)
azis_all_1deg = np.round(azis_all,0)
temps_day_1deg = []
temps_night_1deg = []
counts_1deg = []
for a in range(0,360):
    temps_day_1deg.append(np.nanmean(temps_day[azis_day_1deg == a]))
    temps_night_1deg.append(np.nanmean(temps_night[azis_night_1deg == a]))
    counts_1deg.append(len(azis_all_1deg[azis_all_1deg == a]))

x_day_1deg,y_day_1deg = pol2cart(temps_day_1deg,np.array(range(0,360)),rotate) 
x_night_1deg,y_night_1deg = pol2cart(temps_night_1deg,np.array(range(0,360)),rotate)
x_counts, y_counts = pol2cart(counts_1deg,np.array(range(0,360)),rotate)

In [ ]:





fig, [ax, ax2, ax3] = plt.subplots(1, 3, figsize=(18,5.5))



for each_ax in [ax, ax2]:
    each_ax.plot(0,0,'x',color='grey')
    circle0p5 = plt.Circle((0, 0), radius=0.5, color='grey', fill=False, linestyle='--')
    circle1 = plt.Circle((0, 0), radius=1, color='grey', fill=False, linestyle='--')
    circle1p5 = plt.Circle((0, 0), radius=1.5, color='grey', fill=False, linestyle='--')
    each_ax.add_artist(circle0p5)
    each_ax.add_artist(circle1)
    each_ax.add_artist(circle1p5)


    # SOUTH should be 180
    x_south,y_south = pol2cart(2,190,rotate)
    each_ax.text(x_south,y_south,'SOUTH',fontsize=12)
    
    # EAST should be 90
    x_east,y_east = pol2cart(1.8,90,rotate)
    each_ax.text(x_east,y_east,'EAST',fontsize=12)
    
    # NORTH should be 0
    x_north,y_north = pol2cart(2,350,rotate)
    each_ax.text(x_north,y_north,'NORTH',fontsize=12)
    
    # WEST should be 270
    x_west,y_west = pol2cart(2.3,270,rotate)
    each_ax.text(x_west,y_west ,'WEST',fontsize=12)

    each_ax.text(.35,.35,'0.5$^{\circ}$C',color='grey',fontsize=12)
    each_ax.text(.72,.72,'1.0$^{\circ}$C',color='grey',fontsize=12)
    each_ax.text(1.2,1.2,'1.5$^{\circ}$C',color='grey',fontsize=12)



# DAY
#ax.scatter(x,y,s=5,alpha=0.3,facecolors='tab:gray', edgecolors='none',label='Temperature per 1$^{\circ}$ azimuth bin');
ax.scatter(x_day_1deg,y_day_1deg,s=15,alpha=1,facecolors='k', edgecolors='none',zorder=3,label='Mean Temperature per 1$^{\circ}$ azimuth bin');
#ax.plot(x_1deg,y_1deg,markersize=10,alpha=1, linestyle='-',c='r', zorder=3,label='Mean Temperature per 1$^{\circ}$ azimuth bin');
ax.set_title('Daytime mean bias\nper degree of aspect')

# NIGHT
ax2.scatter(x_night_1deg,y_night_1deg,s=15,alpha=1,facecolors='k', edgecolors='none',zorder=3,label='Mean Temperature per 1$^{\circ}$ azimuth bin');
ax2.set_title('Nighttime')
ax2.set_title('Nighttime mean bias\nper degree of aspect')

# HISTOGRAM (sort of) of AZIMUTHS
#ax3.scatter(x_counts,y_counts,s=15,alpha=1,facecolors='k', edgecolors='none',zorder=3,label='Mean Temperature per 1$^{\circ}$ azimuth bin');
ax3.axis('off')

for each_ax in [ax, ax2]:
    each_ax.set_xlim((-2.5,2.5))
    each_ax.set_ylim((-2.5,2.5))
    each_ax.axis('off')
    # Direction of GOES-16's view is about 122 degrees (when north is 0), but here with south as 0, that is 180+122 = 302
    pointer_loc = 1.9
    length = .3
    sun_x, sun_y = pol2cart(pointer_loc+length,122,rotate)
    sun_x2, sun_y2 = pol2cart(pointer_loc,122,rotate)
    each_ax.quiver(sun_x2,sun_y2,-sun_x2,-sun_y2,color='grey',label='GOES-16 ABI')
    each_ax.plot([sun_x,sun_x2],[sun_y,sun_y2],'-',linewidth='2',color='grey')
    each_ax.text(sun_x+.1, sun_y-.2,'GOES-16 View',fontsize=15,color='grey')

#plt.legend(loc='lower right')
plt.savefig('images/mean_bias_per_degree_aspect_day_night_ASTER-ABI.png',dpi=200)




### Fveg (zeros removed)

In [ ]:
y = ds.fveg.values.ravel()#np.array([ds.fveg.values] * 27).ravel()

# MODIS-ASTER deltaT
plt.figure(figsize=(5,5))
x = ds.ast_mod_mean_diff_tb.values.ravel()
x0 = x[y!=0]
y0 = y[y!=0]
plt.scatter(x0,y0,marker='.',c='k',alpha=0.02)
_x = x0[~np.isnan(x0)]
_y = y0[~np.isnan(x0)]
slope, intercept, r, p, se = stats.linregress(_x, _y)
plt.plot(np.linspace(_x.min(),_x.max()),
         slope*np.linspace(_x.min(),_x.max())+intercept,
         '--r',zorder=99)
plt.ylabel('$F_{veg}$')
plt.xlabel('MODIS-ASTER $\Delta T\,[\degree C]$')
plt.title('MODIS-ASTER $\Delta T\,[\degree C]$\nversus $F_{veg}$')

# ABI-ASTER deltaT
plt.figure(figsize=(5,5))
x = ds.ast_goes_mean_diff_tb.values.ravel()
x0 = x[y!=0]
y0 = y[y!=0]
plt.scatter(x0,y0,marker='.',c='k',alpha=0.02)
_x = x0[~np.isnan(x0)]
_y = y0[~np.isnan(x0)]
slope, intercept, r, p, se = stats.linregress(_x, _y)
plt.plot(np.linspace(_x.min(),_x.max()),
         slope*np.linspace(_x.min(),_x.max())+intercept,
         '--r',zorder=99)
plt.ylabel('$F_{veg}$')
plt.xlabel('ABI-ASTER $\Delta T\,[\degree C]$')
plt.title('ABI-ASTER $\Delta T\,[\degree C]$\nversus $F_{veg}$')

# Absolute value of ABI-ASTER deltaT
plt.figure(figsize=(5,5))
x = np.abs(ds.ast_goes_mean_diff_tb.values.ravel())
x0 = x[y!=0]
y0 = y[y!=0]
plt.scatter(x0,y0,marker='.',c='k',alpha=0.02)
_x = x0[~np.isnan(x0)]
_y = y0[~np.isnan(x0)]
slope, intercept, r, p, se = stats.linregress(_x, _y)
plt.plot(np.linspace(_x.min(),_x.max()),
         slope*np.linspace(_x.min(),_x.max())+intercept,
         '--r',zorder=99)
plt.ylabel('$F_{veg}$')
plt.xlabel('|ABI-ASTER $\Delta T$| $[\degree C]$')
plt.title('|ABI-ASTER $\Delta T$| $[\degree C]$\nversus $F_{veg}$')

In [ ]:
y = ds.fveg.mean(axis=0).values.ravel()

# MODIS-ASTER deltaT
plt.figure(figsize=(5,5))
x = ds.ast_mod_mean_diff_tb.mean(axis=0).values.ravel()
x0 = x[y!=0]
y0 = y[y!=0]
plt.scatter(x0,y0,marker='.',c='k',alpha=0.02,s=5)
_x = x0[~np.isnan(x0)]
_y = y0[~np.isnan(x0)]
slope, intercept, r, p, se = stats.linregress(_x, _y)
plt.plot(np.linspace(_x.min(),_x.max()),
         slope*np.linspace(_x.min(),_x.max())+intercept,
         '--r',zorder=99)
plt.ylabel('$F_{veg}$')
plt.xlabel('MODIS-ASTER $\Delta T\,[\degree C]$')
plt.title('MODIS-ASTER $\Delta T\,[\degree C]$\nversus $F_{veg}$')

# ABI-ASTER deltaT
plt.figure(figsize=(5,5))
x = ds.ast_goes_mean_diff_tb.mean(axis=0).values.ravel()
x0 = x[y!=0]
y0 = y[y!=0]
plt.scatter(x0,y0,marker='.',c='k',alpha=0.02,s=5)
_x = x0[~np.isnan(x0)]
_y = y0[~np.isnan(x0)]
slope, intercept, r, p, se = stats.linregress(_x, _y)
plt.plot(np.linspace(_x.min(),_x.max()),
         slope*np.linspace(_x.min(),_x.max())+intercept,
         '--r',zorder=99)
plt.ylabel('$F_{veg}$')
plt.xlabel('ABI-ASTER $\Delta T\,[\degree C]$')
plt.title('ABI-ASTER $\Delta T\,[\degree C]$\nversus $F_{veg}$')

# Absolute value of ABI-ASTER deltaT
plt.figure(figsize=(5,5))
x = np.abs(ds.ast_goes_mean_diff_tb.mean(axis=0).values.ravel())
x0 = x[y!=0]
y0 = y[y!=0]
plt.scatter(x0,y0,marker='.',c='k',alpha=0.02,s=5)
_x = x0[~np.isnan(x0)]
_y = y0[~np.isnan(x0)]
slope, intercept, r, p, se = stats.linregress(_x, _y)
plt.plot(np.linspace(_x.min(),_x.max()),
         slope*np.linspace(_x.min(),_x.max())+intercept,
         '--r',zorder=99)
plt.ylabel('$F_{veg}$')
plt.xlabel('|ABI-ASTER $\Delta T$| $[\degree C]$')
plt.title('|ABI-ASTER $\Delta T$| $[\degree C]$\nversus $F_{veg}$')

---
---

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np
import random

# dummy variables for demonstration
x = ds.ast_goes_mean_diff_tb.values.ravel()
y = ds.fveg.values.ravel()
z = np.array([hs_sat.values]*27).ravel()

# build the figure instance
fig = plt.figure(figsize=(10,8))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(x, y, z, c='k', marker='.', alpha=0.01)

# set your labels
ax.set_xlabel('deltaT')
ax.set_ylabel('Fveg')
ax.set_zlabel('hs_sat')


ax.view_init(20,135)

plt.show()

---
---

In [ ]:
tree_height = 1
lzas = np.arange(0,90,1)
projected_tree_lengths = tree_height/np.tan(np.radians(90-lzas))
plt.plot(lzas,projected_tree_lengths)